# Rock Piano (ver. 1.0)

## "Music comes from the heart!" ---LU
 
***
 
### Powered by tegridy-tools TMIDIX Optimus Processors: https://github.com/Tegridy-Code/tegridy-tools
 
***
 
### Credit for GPT2-RGA code used in this colab goes out @ Sashmark97 https://github.com/Sashmark97/midigen and @ Damon Gwinn https://github.com/gwinndr/MusicTransformer-Pytorch
 
***
 
### WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/
 
***
 
#### Project Los Angeles
 
#### Tegridy Code 2021
 
***

In [ ]:
!nvidia-smi

In [1]:
#@title Install all dependencies (run only once per session)
    
!git clone https://github.com/asigalov61/Rock-Piano    
    
!pip install tqdm

Cloning into 'Rock-Piano'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 116 (delta 42), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (116/116), 51.11 MiB | 15.86 MiB/s, done.
Resolving deltas: 100% (42/42), done.
Updating files: 100% (22/22), done.


In [3]:
print('Loading needed modules. Please wait...')
import os
from datetime import datetime
import secrets
import copy
import tqdm
from tqdm import tqdm

os.chdir('Rock-Piano')

print('Loading TMIDIX module...')
import TMIDIX
print('Loading GPT2RGA module...')
from GPT2RGA import *

Loading needed modules. Please wait...
Loading TMIDIX module...
Loading GPT2RGA module...
Loading GPT2-RGA Module...
GPT2-RGA loading complete!
Enjoy!


In [4]:
# Unzip the Model and the Training Data

print('Unzipping...')
print('=' * 70)
!unzip -j 'Training-Data/Rock-Piano-Training-Data.zip'

!cat Model/Rock-Piano-Trained-Model.zip* > Rock-Piano-Trained-Model.zip
print('=' * 70)

!unzip -j Rock-Piano-Trained-Model.zip
print('=' * 70)
print('Done!')

Unzipping...
Archive:  Training-Data/Rock-Piano-Training-Data.zip
  inflating: Rock-Piano-Training-Data.pickle  
Archive:  Rock-Piano-Trained-Model.zip
  inflating: Rock-Piano-Trained-Model.pth  
Done!


In [5]:
#@title Load/Reload the model
full_path_to_model_checkpoint = "Rock-Piano-Trained-Model.pth" #@param {type:"string"}

print('Loading the model...')
config = GPTConfig(VOCAB_SIZE, 
                   max_seq,
                   dim_feedforward=dim_feedforward,
                   n_layer=6, 
                   n_head=8, 
                   n_embd=512,
                   enable_rpr=True,
                   er_len=max_seq)

model = GPT(config).to(get_device())

model.load_state_dict(torch.load(full_path_to_model_checkpoint))

print('Done!')

Loading the model...
Done!


In [7]:
# Load the Training Data for priming the model

# Re-running this code will owerwrite the continuation code below

# Re-run this to prime from Training Data at any time

inputs = []

song_ints = []

data = TMIDIX.Tegridy_Any_Pickle_File_Reader('Rock-Piano-Training-Data')


pe = data[0]
for d in tqdm(data):
    song_ints.extend([int(abs(d[1] - pe[1]) / 10)+33, int(d[2] / 10)+33, d[3]+33, d[4]+33, 10])
    pe = d
    
print('Done!')

Tegridy Pickle File Loader
Loading the pickle file. Please wait...


100%|██████████| 3560393/3560393 [00:03<00:00, 937819.91it/s] 

Done!


In [9]:
# Run this code to prime with your own custom MIDI

# MIDI MUST HAVE ONLY PIANO-DRUMS instruments or it may not continue the composition properly

print('Loading your custom MIDI...')

data = TMIDIX.Optimus_MIDI_TXT_Processor('Rock-Piano-Continuation-Seed-1.mid', 
                                         MIDI_channel=16, 
                                         musenet_encoding=True, 
                                         perfect_timings=True)


pe = data[2][0]
for d in tqdm(data[2]):
    inputs.extend([int(abs(d[1] - pe[1]) / 10)+33, int(d[2] / 10)+33, d[3]+33, d[4]+33, 10])
    pe = d
    
print('Done!')

Loading your custom MIDI...


100%|██████████| 582/582 [00:00<00:00, 330367.43it/s]

Done!


In [11]:
#@title Generate Music

number_of_tokens_to_generate = 1024 #@param {type:"slider", min:8, max:1024, step:8}
use_random_primer = False #@param {type:"boolean"}
number_of_ticks_per_quarter = 500 #@param {type:"slider", min:50, max:1000, step:50}
dataset_time_denominator = 10
melody_conditioned_encoding = False
encoding_has_MIDI_channels = False 
encoding_has_velocities = False
simulate_velocity = True #@param {type:"boolean"}
save_only_first_composition = True
chars_encoding_offset_used_for_dataset = 33

fname = 'Rock-Piano-Composition'

print('Rock Piano Model Generator')

output_signature = 'Rock Piano'
song_name = 'RGA Composition'

model.eval()

if use_random_primer:
    sequence = [random.randint(10, 387) for i in range(64)]
    idx = secrets.randbelow(len(sequence))
    rand_seq = model.generate(torch.Tensor(sequence[idx:idx+120]), target_seq_length=number_of_tokens_to_generate)
    out = rand_seq[0].cpu().numpy().tolist()

else:
    out = []
  
    #try:
    if len(inputs) > 0:
      rand_seq = model.generate(torch.Tensor(inputs[-128:]), target_seq_length=number_of_tokens_to_generate)
      out = rand_seq[0].cpu().numpy().tolist()
    else:
      idx = secrets.randbelow(len(song_ints))
      rand_seq = model.generate(torch.Tensor(song_ints[idx:idx+120]), target_seq_length=number_of_tokens_to_generate)
      out = rand_seq[0].cpu().numpy().tolist()
  
  #except:
    #print('=' * 50)
    #print('Error! Try random priming instead!')
    #print('Shutting down...')
    #print('=' * 50)

if len(out) != 0:
  song = []
  sng = []
  for o in out:
    if o != 10:
      sng.append(o)
    else:
      if len(sng) == 4:
        song.append(sng)
      sng = []

  char_offset = 33
  song_f = []
  time = 0
  for s in song:
    
    song_f.append(['note', (abs(time)) * 10, (s[1]-char_offset) * 10, s[2]-char_offset, s[3]-char_offset, s[3]-char_offset])
    time += (s[0] - char_offset)

  detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = output_signature,  
                                                        output_file_name = fname, 
                                                        track_name=song_name, 
                                                        number_of_ticks_per_quarter=number_of_ticks_per_quarter)

  print('Done!')

  #print('Downloading your composition now...')
  #from google.colab import files
  #files.download(fname + '.mid')

  print('=' * 70)
  print('Detailed MIDI stats:')
  for key, value in detailed_stats.items():
        print('=' * 70)
        print(key, '|', value)

  print('=' * 70)

else:
  print('Models output is empty! Check the code...')
  print('Shutting down...')

Rock Piano Model Generator
Generating sequence of max length: 1024
150 / 1024
200 / 1024
250 / 1024
300 / 1024
350 / 1024
400 / 1024
450 / 1024
500 / 1024
550 / 1024
600 / 1024
650 / 1024
700 / 1024
750 / 1024
800 / 1024
850 / 1024
900 / 1024
950 / 1024
1000 / 1024
Converting to MIDI. Please stand-by...
Done! Enjoy! :)
Done!
Detailed MIDI stats:
bank_select | []
channels_by_track | [set(), {0, 9}]
channels_total | {0, 9}
general_midi_mode | []
ntracks | 2
nticks | 7610
num_notes_by_channel | {9: 133, 0: 71}
patch_changes_by_track | [{}, {0: 0, 1: 24, 2: 32, 3: 40, 4: 42, 5: 46, 6: 56, 7: 71, 8: 73, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}]
patch_changes_total | {0, 32, 71, 40, 73, 42, 46, 24, 56}
percussion | {69: 61, 51: 16, 39: 8, 38: 11, 64: 14, 36: 15, 62: 8}
pitches | {50: 4, 67: 7, 62: 17, 59: 9, 71: 1, 52: 1, 53: 1, 64: 3, 55: 7, 60: 2, 66: 3, 69: 3, 43: 5, 57: 4, 54: 4}
pitch_range_by_track | [(0, 0), (43, 71)]
pitch_range_sum | 28
ticks_per_quarter | 500


In [ ]:
#@title Auto-Regressive Generator

#@markdown NOTE: You much generate a seed composition first or it is not going to start

number_of_cycles_to_run = 5 #@param {type:"slider", min:1, max:50, step:1}
number_of_prime_tokens = 128 #@param {type:"slider", min:64, max:256, step:64}

print('=' * 70)
print('Rock Piano Auto-Regressive Model Generator')
print('=' * 70)
print('Starting up...')
print('=' * 70)
print('Prime length:', len(out))
print('Prime tokens:', number_of_prime_tokens)
print('Prime input sequence', out[-8:])

if len(out) != 0:
  print('=' * 70)
  out_all = []
  out_all.append(out)
  for i in tqdm(range(number_of_cycles_to_run)):
      rand_seq1 = model.generate(torch.Tensor(out[-number_of_prime_tokens:]), target_seq_length=1024)
      out1 = rand_seq1[0].cpu().numpy().tolist()
      out_all.append(out1[number_of_prime_tokens:])
      out = out1[number_of_prime_tokens:]
      
      print(chr(10))
      print('=' * 70)
      print('Block number:', i+1)
      print('Composition length so far:', (i+1) * 1024, 'tokens')
      print('=' * 70)

  print('Done!' * 70)
  print('Total blocks:', i+1)
  print('Final omposition length:', (i+1) * 1024, 'tokens')
  print('=' * 70)
  
  out2 = []
  for o in out_all:
    out2.extend(o)

  if len(out2) != 0:
    song = []
    sng = []
    for o in out2:
      if o != 10:
        sng.append(o)
      else:
        if len(sng) == 4:
          song.append(sng)
        sng = []

    char_offset = 33
    song_f = []
    time = 0
    for s in song:
      
        song_f.append(['note', (abs(time)) * 10, (s[1]-char_offset) * 10, s[2]-char_offset, s[3]-char_offset, s[3]-char_offset])      
        time += (s[0] - char_offset)

    song_name = 'Auto-Regressive RGA Composition'

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                          output_signature = output_signature,  
                                                          output_file_name = fname, 
                                                          track_name=song_name, 
                                                          number_of_ticks_per_quarter=number_of_ticks_per_quarter)

    print('Done!')

    #print('Downloading your composition now...')
    #from google.colab import files
    #files.download(fname + '.mid')

    print('=' * 70)
    print('Detailed MIDI stats:')
    for key, value in detailed_stats.items():
          print('=' * 70)
          print(key, '|', value)

    print('=' * 70)

else:
  print('=' * 70)
  print('INPUT ERROR !!!')
  print('Prime sequence is empty...')
  print('Please generate prime sequence and retry')

print('=' * 70)

Rock Piano Auto-Regressive Model Generator
Starting up...
Prime length: 1024
Prime tokens: 128
Prime input sequence [84, 10, 33, 33, 42, 102, 10, 45]


  0%|          | 0/5 [00:00<?, ?it/s]

Generating sequence of max length: 1024
150 / 1024


# Congrats! :) You did it! :)